# Language Modelling (1/3): N-Gram models

TODO:
Download stories one by one and append `END_OF_STORY`, `END_OF_CHAPTER`, `END_OF_PARAGRAPH` tags in appropriate places.

I've scraped every text file on [this page](http://textfiles.com/stories/) and removed newline escape characters. They are random stories written by people of the internet and it amounts to around 12MB of text data. This may not be enough to train a proper model but will do for development purposes.

In [59]:
from utils import load_obj
import nltk

story_text = load_obj('raw_stories')


from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    # Stripping html tags
    s = MLStripper()
    s.feed(html)
    return s.get_data()
print story_text[2000:3000]

def remove_things(text, list_of_things):
    for l in list_of_things:
        text=text.replace(l, '')
    return text

story_text = strip_tags(story_text).lower()
#story_text = story_text.replace('', '.')
story_text = story_text.replace('_', ' _ ')
story_text = story_text.replace('..', '...')
story_text = story_text.replace(' .', '.')
story_text = story_text.replace('. ', ' . ')
story_text = story_text.replace(' ,', ',')
story_text = story_text.replace(' !', '!')
story_text = story_text.replace(' ?', '?')
story_text = story_text.replace('\x00', '')
story_text = remove_things(story_text, ['`', '``', '(', ')'])
print "\n*** REMOVING HTML TAGS *** {} \n".format(type(story_text))
print story_text[2000:3000]
#print story_text[10000:12000]
#print story_text[100000:105000]
print story_text.index('``')

es didn't do it much justice . People did read and download the stories and a few of them encouraged me to write more . So I did . The subsequent stories came solely from my imagination except  " Feqh Galaxy "  and  " Galaxy Sei "  which were both based on games I participated in . "  <P>  " Since I was a kid my writing had a lot of room for improvement . Equally funny as the writing itself is the massive imaginary fan base I addressed in almost every story . The first three stories were clearly an attempt to exalt myself since I felt my deserved glory had been stolen . I also wrote a document about trade and economics in the game itself . This was mostly a theoretical document and doesn't have much practical value . In the years after I wrote all this stuff I actually did discover a few tricks beyond the standard 'two empires at different economic levels trade and make money' strategy . Some day I'll write those down . "  <P>  " SRE is still one of my favorite online games . Unfortuna

ValueError: substring not found

There is some noise to do with titles of stories appearing at the start of each text but there's not much I can do about this.

## Getting word statistics

Now I'll look at the frequencies of words and think about whether to limit the vocabulary size

In [57]:
try:
    print story_text.index('``')
except:
    print "No backticks found before tokenizing"
import nltk
from utils import sequence_stats
# Splits up text into sentences (accounts for ! and ?)
sent_text = nltk.sent_tokenize(story_text) 
# Splits up sequences into tokens
tok_text = [nltk.word_tokenize(w) for w in sent_text] 

text =[' '.join(s) for s in tok_text]
print text[0:1000]
flat = []
for s in tok_text:
    flat+=s

freq = nltk.FreqDist(flat)

def covered_with_vocab(freq_dist, vocab_size):
    total_words = len(freq_dist.keys())
    print "There are {} words in the full vocabulary.".format(total_words)
    total_length = sum(freq_dist.values())
    print "There are {} hapax legomenon.".format(len(freq_dist.hapaxes()))
    print "Vocab size of {} restricts to {:0.2f} percent of total vocab.".format(
                                                vocab_size, 100*(float(vocab_size)/total_words))
    vocab = dict(freq_dist.most_common(vocab_size))
    #print "vocab size= ",len(vocab.keys())
    covered_with_vocab = sum(vocab.values()) # Total of frequencies
    print "Percentage of whole corpus covered by vocab: {:0.2f}".format(
                                                    100*(covered_with_vocab/float(total_length)))

covered_with_vocab(freq, 10000)
sequence_stats(tok_text)


No backticks found before tokenizing
[u't e x t f i l e s stories : tristan farnon filename size description of the textfile at the request of tristan farnon this directory has been removed .', u'this saddens me no end because his stories were truly wonderful .', u"he 'd post them on the dark side of the moon ae and while there were always interesting files being posted there tristan 's always caused a rush of excitement through the place .", u"you could tell he 'd posted one because the system would be really busy .", u"tristan 's stories were filled with irony interesting situations and well-written dialogue all pleasant changes from what else you would find .", u"they were n't interested in causing destruction or insulting others ; they just entertained .", u'tristan farnon has gone on to do excellent web-based work on his site leisuretown which continues to display his excellent sense of humor irony characterization and flat-out quality .', u'i give it my highest recommendation .',

In [39]:

SOS_ID = 3
UNK_ID = 2
EOS_ID = 1
PAD_ID = 0
static_IDs = [SOS_ID, UNK_ID, EOS_ID, PAD_ID]
def word_to_ids(freq_dist, vocab_size):
    #vocab = dict(freq_dist.most_common(vocab_size))
    s = sorted(freq_dist.most_common(vocab_size), key=lambda x: x[1], reverse=True)
    # Remove word from vocab if it occurs fewer than 4 times
    vocab = [w for w, i in s if i >4]
    
    #vocab = zip(*)
    vocab_size = len(vocab)
    # IDs begin at 4 because <EOS>=1, <UNK>=2 and <SOS>=3
    word_to_ids = dict([(word, i+max(static_IDs)+1) for i, word in enumerate(vocab)])
    word_to_ids[u'<SOS>'] = SOS_ID
    word_to_ids[u'<UNK>'] = UNK_ID
    word_to_ids[u'<EOS>'] = EOS_ID
    word_to_ids[u'<PAD>'] = PAD_ID
    id_to_words = dict((idx, word) for word, idx in word_to_ids.items())
    return word_to_ids, id_to_words, vocab_size

word2id, id2word, vocab_size = word_to_ids(freq, 40000)
covered_with_vocab(freq, vocab_size)

There are 59987 words in the full vocabulary.
There are 26822 hapax legomenon.
Vocab size of 18253 restricts to 30.43 percent of total vocab.
Percentage of whole corpus covered by vocab: 97.20


In [40]:
stopp = [w for w in word2id.keys() if '.' in w]
print len(stopp)

149


In [41]:
print word2id['``']

6


In [25]:
print len(tok_text)

160783


## Swapping words for IDs

Now I go through the text and swap out the words for the ID integers.

In [26]:
from multiprocessing import Pool
import time
from functools import partial
def replace_id_func(sequence, **kwargs):
    # Parallelisable version of word to ID replacer
    word2id=kwargs['wtid']
    ids_sent = [] # sentence with words replaced by ids
    sequence = [u'<SOS>']+sequence+[u'<EOS>']
    for token in sequence:
        if token not in word2id.keys():
            if token == u'<EOS>':
                ids_sent.append(1)
            else:
                ids_sent.append(2)
        else:
            ids_sent.append(word2id[token])
    return ids_sent

def replace_with_word_id(text, wtid):
    '''
    take the list of lists, find FreqDist, replace any
    out of vocabulary words with <UNK> whilst giving each
    token a numerical ID, return new list of lists'''
    ti = time.time()
    prep_text = []
    pool = Pool(processes = 3)
    kwargs = {'wtid': wtid}
    # Parallelise process, set buffer size to restrict memory usage
    id_text = [i for i in pool.imap(partial(replace_id_func, **kwargs), text, chunksize=30000)]
    print "Replacing words with IDs took {} seconds.".format(time.time()-ti)
    return id_text

text_ids = replace_with_word_id(tok_text[0:60000], word2id)

Replacing words with IDs took 245.14815402 seconds.


In [6]:
from utils import load_obj
data_dic = load_obj('data_dic')
X_train = data_dic['X_train']
y_train = data_dic['y_train']
word2id = data_dic['word2id']
id2word = data_dic['id2word']

Now we can calculate the unigram conditional probability distributions to answer the question: "which words are likely to come after the current word?"

In [98]:
import time
import itertools
EOS_ID = 3 # End of chapter/story token
SOS_ID = 2 # Start of chapter/story
UNK_ID = 1 # Unknown word (out of vocabulary)
PAD_ID = 0 # 
#{PAD_ID:{PAD_ID:1}, EOS_ID:{PAD_ID:1}}

def get_unigram_prob(text, target):
    # Returns a dictionary whose keys are current words and values are
    # dictionaries with possible next words and their probabilities
    ti = time.time()
    vocab = list(set(itertools.chain(*text)))
    unigram_prob = dict((v, dict()) for v in vocab)
    
    for X,y in zip(text, target):
        for i in range(len(X)):
            w_x = X[i]
            # If it's not an EOS token, count the next unigram
            w_y = y[i]
            if w_y == UNK_ID:
                pass
            else:                
                if w_y not in unigram_prob[w_x].keys():
                        unigram_prob[w_x][w_y] = 1
                else:
                    unigram_prob[w_x][w_y] += 1
    #unigram_prob.update({PAD_ID:{PAD_ID:1}, EOS_ID:{PAD_ID:1}})
    # We've got the frequencies, now normalise
    for k in unigram_prob.keys():
        norm = float(sum(unigram_prob[k].values()))
        for kk, v in unigram_prob[k].items():
            unigram_prob[k].update({kk: v/norm})
    
    print "Calculating Unigram probabilities took {} seconds.".format(time.time()-ti)
    return unigram_prob
unigram_test = get_unigram_prob([[SOS_ID, 5,6,7,8,9]],[[5,6,7,8,9,EOS_ID]])
exp = {2: {5 : 1.}, 5: {6 : 1.},6: {7 : 1.0},7: {8 : 1.}, 8: {9 : 1.},9: {3 : 1.}}
print "EXPECTED = {}".format(exp)
print "ACTUALS = {}".format(unigram_test)
for k in unigram_test.keys():
    print k, unigram_test[k],exp[k]

Calculating Unigram probabilities took 4.41074371338e-05 seconds.
EXPECTED = {2: {5: 1.0}, 5: {6: 1.0}, 6: {7: 1.0}, 7: {8: 1.0}, 8: {9: 1.0}, 9: {3: 1.0}}
ACTUALS = {2: {5: 1.0}, 5: {6: 1.0}, 6: {7: 1.0}, 7: {8: 1.0}, 8: {9: 1.0}, 9: {3: 1.0}}
2 {5: 1.0} {5: 1.0}
5 {6: 1.0} {6: 1.0}
6 {7: 1.0} {7: 1.0}
7 {8: 1.0} {8: 1.0}
8 {9: 1.0} {9: 1.0}
9 {3: 1.0} {3: 1.0}


In [99]:

unigram_prob = get_unigram_prob(X_train, y_train)

Calculating Unigram probabilities took 5.74447393417 seconds.


Using these probabilities, we can predict what the most likely word to come next would be...

In [100]:
import operator
from utils import ids_to_phrases
def ids_to_phrases(idx_list, id_to_word, stop_IDs=[0,3]):
    # Takes list of word ids and returns a string of words
    # Mainly for use in analysis
    phrase = u''
    i=0
    if len(idx_list)>0:
        "We want to ignore <EOS> unless it's the first element"
        while len(idx_list)>i and (i>=0) and (idx_list[i] not in stop_IDs):
            phrase+= id_to_word[idx_list[i]]+u' '
            i+=1
    return phrase.encode("utf-8")

#print unigram_prob[2]
def greedy_argmax(seq):
    # Just chooses the most likely next word
    current_id = seq[-1]
    next_id = max(unigram_prob[current_id].iteritems(), key=operator.itemgetter(1))[0]
    return next_id

def sample_creator(f, start='<SOS>', length = 10):
    sample = [word2id[start]]
    for i in range(length):
        sample.append(f(sample))
    return sample

print "Sample output with greedy argmax choice: "
print '\n'.join([ids_to_phrases(sample_creator(greedy_argmax), id2word) for i in range(3)])
print id2word[(greedy_argmax([word2id['the']]))]

Sample output with greedy argmax choice: 
<SOS> the king 's son , and the king 's son 
<SOS> the king 's son , and the king 's son 
<SOS> the king 's son , and the king 's son 
king


Making greedy predictions (just choosing the most likely token) is a terrible idea. It makes for very boring sequences, dominated by `<UNK>` and common filler words. We would now like to choose words from a probability distribution of how likely they are.

In [159]:
import random

def weighted_choice(inner_dict):
    """items is a list of tuples in the form (item, weight)"""
    n = random.uniform(0, 1)
    for item, weight in inner_dict.items():
        if n < weight:
            return item
        n = n - weight
    try:
        return item
    except:
        return random.randint(0,8000)

s = [weighted_choice(unigram_prob[SOS_ID]) for s in range(10)]

print "LIKELY SENTENCE BEGINNERS: \n"+' '.join([id2word[i] for i in s])
print word2id['``']

LIKELY SENTENCE BEGINNERS: 
king the the i there the in lullaby the the
13


In [150]:
def unigram_greedy_sample(seq):
    current_id = seq[-1]
    next_id = weighted_choice(unigram_prob[current_id])
    return next_id

print "Sample text with greedy probabilistic choice: "
print '\n'.join([ids_to_phrases(sample_creator(unigram_greedy_sample), id2word) for i in range(10)])

Sample text with greedy probabilistic choice: 
<SOS> tree , and at it for you must be happy 
<SOS> the old man who had plowed away to be married 
<SOS> the people , with mud the tree , the door 
<SOS> the princess . the cat , they can ride one 
<SOS> `` go as it mean that killed by , said 
<SOS> there 's room , and thought of the learned to 
<SOS> there was shining with her company tremble to-morrow ! ’ 
<SOS> there , and the man was amazed to destroy them 
<SOS> king 's treasury , and say kindly they resolved to 
<SOS> how are thoughtful never quite exhausted by the morning when 


We can see, adding randomness to the choice algorithm improves the output substantially. We could see if this is improved further by using beam search, however, the core problem still remains that the sequences have no memory of their past beyond the previously generated token. This often leads to nonsensical text and it would be impossible for long-term dependencies to be expressed. 

We can improve this by using N-Gram probabilities. For example, choosing a value of N=2, we could build a dictionary whose keys are the two-word combinations which appear together in the text and whose values are sub-dictionaries of all the words which follow the bigrams and their respective probabilities. Then when making predictions on a current sequence, we check to see if there have been already been two words - if so, see if they form a recognised _bigram_ and find out which unigram to choose next. If not, just use the unigram method as before.

In [151]:

def get_Ngram_prob(text, target, N=2):
    # Returns a dictionary whose keys are current words and values are
    # dictionaries with possible next words and their probabilities
    ti = time.time()
    vocab = id2word.keys()
    Ngrams = ['-'.join(str(ii) for ii in X[i:i+N]) for X,y in zip(text, target) for i in range(len(X)-N+1)]
    
    Ngram_prob = dict((ng, dict()) for ng in Ngrams)
    count=0
    for X,y in zip(text, target):
        if count%100==0:
            print "{}gram prob. dictionary {:0.3f} percent finished".format(N, (float(count)/len(text))*100)
        count +=1
        for i in range(len(X)-N+1):
            w_x = '-'.join(str(ii) for ii in X[i:i+N])
            # If it's not an EOS token, count the next unigram
            w_y = y[i+N-1]
            if w_y == UNK_ID:
                pass
            else:
                if w_y not in Ngram_prob[w_x].keys():
                    Ngram_prob[w_x][w_y] = 1
                else:
                    Ngram_prob[w_x][w_y] += 1
    # We've got the frequencies, now normalise
    for k in Ngram_prob.keys():
        norm = float(sum(Ngram_prob[k].values()))
        for kk, v in Ngram_prob[k].items():
            Ngram_prob[k].update({kk: v/norm})
    print "Calculating Ngram probabilities took {} seconds.".format(time.time()-ti)
    return Ngram_prob
"""
def Ngram_prob(text, N):
    # Returns a dictionary whose keys are current words and values are
    # dictionaries with possible next words and their probabilities
    ti = time.time()
    Ngram_prob = dict()
    count = 0.
    for seq in text:
        if count%100==0:
            print "{}gram prob. dictionary {} percent finished".format(N, (count/len(text))*100)
        count +=1
        # len(seq)-N because we only want to check up to the final word
        for i in range(len(seq)-N):
            # what are the N next words?
            w_t = '-'.join(str(ii) for ii in seq[i:i+N])
            # If it's not an EOS token, count the next unigram
            try:
                # What is the word after the N next words?
                w_tt = seq[i+N]
                if w_t not in Ngram_prob.keys():
                    Ngram_prob[w_t] = {w_tt: 1}
                else:
                    if w_tt != 2:
                        if w_tt not in Ngram_prob[w_t].keys():
                                Ngram_prob[w_t][w_tt] = 1
                        else:
                            Ngram_prob[w_t][w_tt] += 1
            except IndexError:
                Ngram_prob[w_t] ={PAD_ID: 1}
    # We've got the frequencies, now normalise
    for k in Ngram_prob.keys():
        norm = float(sum(Ngram_prob[k].values()))
        for kk, v in Ngram_prob[k].items():
            Ngram_prob[k].update({kk: v/norm})
    print "Calculating {}gram probabilities took {} seconds.".format(N, time.time()-ti)
    return Ngram_prob
"""
bigram_test = get_Ngram_prob([[SOS_ID, 5,6,7,8,9]],[[5,6,7,8,9,EOS_ID]], N=2)
exp = {str(SOS_ID)+'-5': {6 : 1.}, '5-6': {7 : 1.},'6-7': {8 : 1.0},'7-8': {9 : 1.0},'8-9': {str(EOS_ID) : 1.}}
print "EXPECTED = {}".format(exp)
print "Predicted = {}".format(bigram_test)

2gram prob. dictionary 0.000 percent finished
Calculating Ngram probabilities took 0.000908136367798 seconds.
EXPECTED = {'6-7': {8: 1.0}, '5-6': {7: 1.0}, '7-8': {9: 1.0}, '8-9': {'3': 1.0}, '2-5': {6: 1.0}}
Predicted = {'6-7': {8: 1.0}, '5-6': {7: 1.0}, '7-8': {9: 1.0}, '8-9': {3: 1.0}, '2-5': {6: 1.0}}


In [152]:
bigram_prob = get_Ngram_prob(X_train, y_train, N=2)

2gram prob. dictionary 0.000 percent finished
2gram prob. dictionary 21.053 percent finished
2gram prob. dictionary 42.105 percent finished
2gram prob. dictionary 63.158 percent finished
2gram prob. dictionary 84.211 percent finished
Calculating Ngram probabilities took 5.70813703537 seconds.


In [153]:

def bigram_greedy_sample(seq):
    # seq will be a variable list of integers
    inner_dict = 0
    try:
        w_t = '-'.join(str(ii) for ii in seq[-2::])
        out = bigram_prob[w_t]
    except:
        out = unigram_prob[seq[-1]]
    next_id = weighted_choice(out)    
    return next_id

bigram_greedy_sample([4,5,6,7,8])


17

In [154]:
print "\n BIGRAMS"
print '\n'.join([ids_to_phrases(sample_creator(bigram_greedy_sample), id2word) for i in range(10)])


 BIGRAMS
<SOS> there lived in the morning , stitching away . it 
<SOS> hearing these words , nor break the lines . a 
<SOS> the death of cormoran , jack put the house by 
<SOS> a tailor ? ' said the eldest leading the way 
<SOS> sweet and sour beer . '' and then two eyes 
<SOS> king grisly-beard , ' said the carp touched the old 
<SOS> `` please , my mabel , this new-made pat of 
<SOS> the boy came down and there a long time . 
<SOS> a good way and seeing himself suddenly surrounded by a 
<SOS> the white one fluttered on to his majesty , with 


Calculating the Bigram probability distributions seems to have been worth it. The generated text is far more fluent now, but still a bit fragmented. Quotation marks, which are denoted by "``", are often opened or closed but rarely both - due to the lack of memory in the model.

Quite amazingly some short sequences make complete sense such as "the fox led the goat with a dignified air . " which I think is a beautiful sentence.

In [155]:
trigram_test = get_Ngram_prob([[SOS_ID, 5,6,7,8,9]],[[5,6,7,8,9,EOS_ID]], N=3)
exp = {str(SOS_ID)+'-5-6': {7 : 1.}, '5-6-7': {8 : 1.},'6-7-8': {9 : 1.0},'7-8-9': {str(EOS_ID) : 1.}}
print "EXPECTED = {}".format(exp)
print "Predicted = {}".format(trigram_test)

trigram_prob = get_Ngram_prob(X_train,y_train, N=3)




3gram prob. dictionary 0.000 percent finished
Calculating Ngram probabilities took 0.000842094421387 seconds.
EXPECTED = {'2-5-6': {7: 1.0}, '7-8-9': {'3': 1.0}, '6-7-8': {9: 1.0}, '5-6-7': {8: 1.0}}
Predicted = {'2-5-6': {7: 1.0}, '7-8-9': {3: 1.0}, '6-7-8': {9: 1.0}, '5-6-7': {8: 1.0}}
3gram prob. dictionary 0.000 percent finished
3gram prob. dictionary 21.053 percent finished
3gram prob. dictionary 42.105 percent finished
3gram prob. dictionary 63.158 percent finished
3gram prob. dictionary 84.211 percent finished
Calculating Ngram probabilities took 5.86414599419 seconds.


In [161]:
def trigram_greedy_pred(seq):
    # seq will be a variable list of integers
    inner_dict = 0
    out = 0
    if len(seq)>0:
        try:
            # Try to find a trigram
            w_t = '-'.join(str(ii) for ii in seq[-3::])
            out = trigram_prob[w_t]
        except:
            try:
                w_t = '-'.join(str(ii) for ii in seq[-2::])
                out = bigram_prob[w_t]
            except:
                print seq[-1]
                out = unigram_prob[seq[-1]]
        next_id = weighted_choice(out)    
        return next_id
    else:
        return SOS_ID

print "\nTRIGRAMS"
print '\n'.join([ids_to_phrases(sample_creator(trigram_greedy_pred, length = 20), id2word) for i in range(20)])


TRIGRAMS
2
2
2
2
2
2
2
2
2
2
3664
2
2
2
2
2
6764
2
2
4736
2
2
2
6307
<SOS> `` arise , for the sake of picciola he would do . but i know you will not want to 
<SOS> cat-skin there was once upon a time , another woman arose and said , 'this lady is dancing with me 
<SOS> pinocchio set out and as frederick walked the fastest , he left his master , who the next morning the 
<SOS> and when he had drawn from the instrument . the little tailor . 'do you stay behind , ' said 
<SOS> a man and quite as broad . of all the palace , where he left her , she will change 
<SOS> androcles a slave named androcles once escaped from his master 's bidding . he was too frightened . but she 
<SOS> the lost doll i once had a sweet little doll , dears , as i am . '' `` ah 
<SOS> once upon a time a queen to whom god had given no children . the children watched it and saw 
<SOS> there was once a shoemaker , who worked very hard and was very beautiful and her mother loved her dearly 
<SOS> sweetheart roland ther